# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 58 new papers today
          16 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/15 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2303.10175


extracting tarball to tmp_2303.10175... done.
Retrieving document from  https://arxiv.org/e-print/2303.10200


extracting tarball to tmp_2303.10200...

 done.
Retrieving document from  https://arxiv.org/e-print/2303.10273


...

 done.
Retrieving document from  https://arxiv.org/e-print/2303.11031


extracting tarball to tmp_2303.11031...

 done.
Retrieving document from  https://arxiv.org/e-print/2303.11035


extracting tarball to tmp_2303.11035...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.10175-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.10175) | **Measuring galaxy cluster mass profiles into low acceleration regions  with galaxy kinematics**  |
|| <mark>Pengfei Li</mark>, et al. |
|*Appeared on*| *2023-03-21*|
|*Comments*| *13 pages, 8 figures, 1 table. Submitted to A&A*|
|**Abstract**| We probe the outer dynamical mass profile of 16 galaxy clusters from the HIghest X-ray FLUx Galaxy Cluster Sample (HIFLUGCS) using galaxy kinematics. Numerically solving the spherical Jeans equation, we parameterize the dynamical mass profile and the galaxy velocity anisotropy profile using two general functions to ensure that our results are not biased towards any specific model. The mass-velocity anisotropy degeneracy is broken by using two "virial shape parameters" that depend on the fourth moment of velocity distribution. The resulting velocity anisotropy estimates consistently show a nearly isotropic distribution in the inner regions, with an increasing radial anisotropy towards large radii. We compare our derived dynamical masses with those calculated from X-ray gas data assuming hydrostatic equilibrium, finding that massive and rich relaxed clusters generally present consistent mass measurements, while unrelaxed or low-richness clusters have systematically larger total mass than hydrostatic mass by an average of 50\%. This might help alleviate current tensions in the measurement of $\sigma_8$, but it also leads to cluster baryon fractions below the cosmic value. Finally, our approach probes accelerations as low as $10^{-11}$ m s$^{-2}$, comparable to the outskirts of individual late-type galaxies. We confirm that galaxy clusters deviate from the radial acceleration relation defined by galaxies. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.10200-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.10200) | **HIP 67506 C: MagAO-X Confirmation of a New Low-Mass Stellar Companion to  HIP 67506 A**  |
|| Logan A. Pearce, et al. -- incl., <mark>Joseph D. Long</mark> |
|*Appeared on*| *2023-03-21*|
|*Comments*| *10 pages, 9 figures, 4 tables, accepted to MNRAS*|
|**Abstract**| We report the confirmation of HIP 67506 C, a new stellar companion to HIP 67506 A. We previously reported a candidate signal at 2$\lambda$/D (240~mas) in L$^{\prime}$ in MagAO/Clio imaging using the binary differential imaging technique. Several additional indirect signals showed that the candidate signal merited follow-up: significant astrometric acceleration in Gaia DR3, Hipparcos-Gaia proper motion anomaly, and overluminosity compared to single main sequence stars. We confirmed the companion, HIP 67506 C, at 0.1" with MagAO-X in April, 2022. We characterized HIP 67506 C MagAO-X photometry and astrometry, and estimated spectral type K7-M2; we also re-evaluated HIP 67506 A in light of the close companion. Additionally we show that a previously identified 9" companion, HIP 67506 B, is a much further distant unassociated background star. We also discuss the utility of indirect signposts in identifying small inner working angle candidate companions. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.10273-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.10273) | **Broadband plasma spray anti-reflection coating technology for  millimeter-wave astrophysics**  |
|| Oliver Jeong, et al. -- incl., <mark>Adrian T. Lee</mark> |
|*Appeared on*| *2023-03-21*|
|*Comments*| **|
|**Abstract**| We present a broadband plasma spray anti-reflection (AR) coating technology for millimeter-wave astrophysics experiments with large-format, cryogenic optics. By plasma spraying alumina- and silica-based powders, we have produced coatings of tunable index of refraction and thickness, low loss, and coefficient of thermal expansion matched to alumina substrates. We demonstrate two-layer AR coatings on alumina with reflection below 5% over 82% and 69% fractional bandwidths for 90/150 and 220/280 GHz passband designs, respectively, and band-averaged absorption loss reduced to $\sim$1\% at 100 K for both AR coatings. We describe the design, tolerance, fabrication process, and optical measurements of these AR coatings. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.10982-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.10982) | **In search for infalling clumps in molecular clouds -- A catalogue of CO  blue-profiles**  |
|| Zhibo Jiang, et al. -- incl., <mark>Shaobo Zhang</mark> |
|*Appeared on*| *2023-03-21*|
|*Comments*| **|
|**Abstract**| We have started a systematic survey of molecular clumps with infall motions to study the very early phase of star formation. Our first step is to utilize the data products by MWISP to make an unbiased survey for blue asymmetric line profiles of CO isotopical molecules. Within a total area of $\sim$ 2400 square degrees nearby the Galactic plane, we have found 3533 candidates showing blue-profiles, in which 3329 are selected from the $^{12}$CO&$^{13}$CO pair and 204 are from the $^{13}$CO&C$^{18}$O pair. Exploration of the parametric spaces suggests our samples are in the cold phase with relatively high column densities ready for star formation. Analysis of the spatial distribution of our samples suggests that they exist virtually in all major components of the Galaxy. The vertical distribution suggest that the sources are located mainly in the thick disk of $\sim$ 85 parsec, but still a small part are located far beyond Galactic midplane. Our follow-up observation indicates that these candidates are a good sample to start a search for infall motions, and to study the condition of very early phase of star formation. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.11001-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.11001) | **Slow solar wind sources. High-resolution observations with a quadrature  view**  |
|| Krzysztof Barczynski, et al. -- incl., <mark>Phil Smith</mark> |
|*Appeared on*| *2023-03-21*|
|*Comments*| *10 pages, 6 figures, accepted for publication in A&A; manuscript is a part of Astronomy & Astrophysics special issue: Solar Orbiter First Results (Nominal Mission Phase)*|
|**Abstract**| The origin of the slow solar wind is still an open issue. One possibility that has been suggested is that upflows at the edge of an active region can contribute to the slow solar wind. We aim to explain how the plasma upflows are generated, which mechanisms are responsible for them, and what the upflow region topology looks like. We investigated an upflow region using imaging data with the unprecedented temporal (3s) and spatial (2 pixels = 236km) resolution that were obtained on 30 March 2022 with the 174{\AA} of the Extreme-Ultraviolet Imager (EUI)/High Resolution Imager (HRI) on board Solar Orbiter. During this time, the EUI and Earth-orbiting satellites (Solar Dynamics Observatory, Hinode, and the Interface Region Imaging Spectrograph, IRIS) were located in quadrature (92 degrees), which provides a stereoscopic view with high resolution. We used the Hinode/EIS (Fe XII) spectroscopic data to find coronal upflow regions in the active region. The IRIS slit-jaw imager provides a high-resolution view of the transition region and chromosphere. For the first time, we have data that provide a quadrature view of a coronal upflow region with high spatial resolution. We found extended loops rooted in a coronal upflow region. Plasma upflows at the footpoints of extended loops determined spectroscopically through the Doppler shift are similar to the apparent upward motions seen through imaging in quadrature. The dynamics of small-scale structures in the upflow region can be used to identify two mechanisms of the plasma upflow: Mechanism I is reconnection of the hot coronal loops with open magnetic field lines in the solar corona, and mechanism II is reconnection of the small chromospheric loops with open magnetic field lines in the chromosphere or transition region. We identified the locations in which mechanisms I and II work. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.11031-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.11031) | **Characterizing the ELG luminosity functions in the nearby Universe**  |
|| G. Favole, et al. -- incl., <mark>H. Guo</mark> |
|*Appeared on*| *2023-03-21*|
|*Comments*| **|
|**Abstract**| Nebular emission lines are powerful diagnostics for the physical processes at play in galaxy formation and evolution. Moreover, emission-line galaxies (ELGs) are one of the main targets of current and forthcoming spectroscopic cosmological surveys. We investigate the contributions to the line luminosity functions (LFs) of different galaxy populations in the local Universe, providing a benchmark for future surveys of earlier cosmic epochs. The large statistics of the observations from the SDSS DR7 Main galaxy sample and the MPA-JHU spectral catalogue enables us to precisely measure the H$\alpha$, H$\beta$, [OII], [OIII], [NII], and [SII] emission-line LFs over ~2.4 Gyrs in the low-z Universe, 0.02<z<0.22. We present a generalised 1/Vmax LF estimator capable of simultaneously correcting for spectroscopic, r-band magnitude, and emission-line incompleteness. We study the contribution to the LF of different types of ELGs classified using two methods: (i) the specific star formation rate (sSFR), and (ii) the line ratios on the Baldwin-Phillips-Terlevich (BPT) and WHAN (i.e. H$\alpha$ equivalent width versus the [NII]/H$\alpha$ ratio) diagrams. The ELGs in our sample are mostly star forming, with 83.6 per cent having sSFR>10$^{-11}$/yr. When classifying ELGs using the BPT+WHAN diagrams, we find that 63 per cent are star forming, 1.5 are passively evolving, and 2.9 have nuclear activity (Seyfert). The rest are LINERs and composite ELGs. We find that a Saunders function is the most appropriate to describe all the emission-line LFs. They are dominated by star-forming regions, except for the bright end of the [OIII] and [NII] LFs (i.e. L[NII]>10$^{42}$ erg/s, L[OIII]>10$^{43}$ erg/s), where the contribution of Seyfert galaxies is not negligible. Besides the star-forming population, composite galaxies and LINERs are the ones that contribute the most to the ELG numbers at L<10$^{41}$ erg/s. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.11035-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.11035) | **Total Electron Temperature Derived from Quasi-Thermal Noise Spectroscopy  In the Pristine Solar Wind: Parker Solar Probe Observations**  |
|| M. Liu, et al. -- incl., <mark>N. Meyer-Vernet</mark> |
|*Appeared on*| *2023-03-21*|
|*Comments*| *10 pages, 7 figures, and Astronomy & Astrophysics Accepted*|
|**Abstract**| The Quasi-thermal noise (QTN) technique is a reliable tool to yield accurate measurements of the electron parameters in the solar wind. We apply this method on Parker Solar Probe (PSP) observations to derive the total electron temperature ($T_e$) from the linear fit of the high-frequency part of the QTN spectra acquired by the RFS/FIELDS instrument, and present a combination of 12-day period of observations around each perihelion from Encounter One (E01) to Ten (E10) (with E08 not included) with the heliocentric distance varying from about 13 to 60 solar radii ($R_\odot{}$). We find that the total electron temperature decreases with the distance as $\sim$$R^{-0.66}$, which is much slower than adiabatic. The extrapolated $T_e$ based on PSP observations is consistent with the exospheric solar wind model prediction at $\sim$10 $R_\odot{}$, Helios observations at $\sim$0.3 AU and Wind observations at 1 AU. Also, $T_e$, extrapolated back to 10 $R_\odot{}$, is almost the same as the strahl electron temperature $T_s$ (measured by SPAN-E) which is considered to be closely related to or even almost equal to the coronal electron temperature. Furthermore, the radial $T_e$ profiles in the slower solar wind (or flux tube with larger mass flux) are steeper than those in the faster solar wind (or flux tube with smaller mass flux). More pronounced anticorrelated $V_p$-$T_e$ is observed when the solar wind is slower and closer to the Sun. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.10410-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.10410) | **Assessing inner boundary conditions for global coronal modeling of solar  maxima**  |
|| Michaela Brchnelova, et al. -- incl., <mark>Fan Zhang</mark> |
|*Appeared on*| *2023-03-21*|
|*Comments*| *11 pages, 4 figures*|
|**Abstract**| Computational Fluid Dynamics (CFD)-based global solar coronal simulations are slowly making their way into the space weather modeling toolchains to replace the semi-empirical methods such as the Wang-Sheeley-Arge (WSA) model. However, since they are based on CFD, if the assumptions in them are too strong, these codes might experience issues with convergence and unphysical solutions. Particularly the magnetograms corresponding to solar maxima can pose problems as they contain active regions with strong magnetic fields, resulting in large gradients. Combined with the approximate way in which the inner boundary is often treated, this can lead to non-physical features or even a complete divergence of the simulation in these cases. Here, we show some of the possible approaches to handle this inner boundary in our global coronal model COolfluid COrona uNstrUcTured (COCONUT) in a way that improves both convergence and accuracy. Since we know that prescribing the photospheric magnetic field for a region that represents the lower corona is not entirely physical, first, we look at the ways in which we can adjust the input magnetograms to remove the highest magnitudes and gradients. Secondly, since in the default setup we also assume a constant density, here we experiment with changing these values locally and globally to see the effect on the results. We conclude, through comparison with observations and convergence analysis, that modifying the density locally in active regions is the best way to improve the performance both in terms of convergence and physical accuracy from the tested approaches. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

200  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

5  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

0  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
